## Sources:
- <https://data.world/lpetrocelli/czech-financial-dataset-real-anonymized-transactions/workspace/intro?fbclid=IwAR1fp9fb023Rq4urPTj1Q7Mebe6ihCf9Uz0lGIjdci6Whq6t_BcPGoDDJJY>
- <https://webpages.uncc.edu/mirsad/itcs6265/group1/loan_preprocessing.html>

## Libraries

In [1]:
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime
from geopy.geocoders import Nominatim

## Account info

In [2]:
# load data
df_account = pd.read_csv(r"Data\csv\account.csv", sep = ';', low_memory=False)

# translation of values
df_account.frequency = df_account.frequency.replace({'POPLATEK MESICNE': 'monthly',
                                                    'POPLATEK TYDNE': 'weekly',
                                                    'POPLATEK PO OBRATU': 'after_transaction'})

# date conversion
df_account.date = pd.to_datetime(df_account.date, format='%y%m%d')

# rename columns
df_account = df_account.rename({'account_creation_date': 'date_account_creation',
                                'frequency': 'issuance_frequency',
                                'date': 'account_creation_date'}, axis=1)

df_account

,account_id,district_id,issuance_frequency,account_creation_date
0,576,55,monthly,1993-01-01
1,3818,74,monthly,1993-01-01
2,704,55,monthly,1993-01-01
3,2378,16,monthly,1993-01-01
4,2632,24,monthly,1993-01-02
...,...,...,...,...
4495,124,55,monthly,1997-12-28
4496,3958,59,monthly,1997-12-28
4497,777,30,monthly,1997-12-28
4498,1573,63,monthly,1997-12-29


## Card

In [3]:
# load data 
df_card = pd.read_csv(r"Data\csv\card.csv", sep = ';', low_memory=False)

# date conversion
df_card.issued = pd.to_datetime(df_card.issued.astype(str).str[:6], format='%y%m%d')

# rename columns
df_card = df_card.rename({'disp_id': 'disposition_id',
                          'type': 'card_type',
                          'issued': 'card_issue_date'}, axis=1) 

df_card

,card_id,disposition_id,card_type,card_issue_date
0,1005,9285,classic,1993-11-07
1,104,588,classic,1994-01-19
2,747,4915,classic,1994-02-05
3,70,439,classic,1994-02-08
4,577,3687,classic,1994-02-15
...,...,...,...,...
887,125,694,gold,1998-12-26
888,674,4360,classic,1998-12-28
889,322,2063,classic,1998-12-28
890,685,4467,classic,1998-12-29


## Clients

In [4]:
# load data
df_client = pd.read_csv(r"Data\csv\client.csv", sep = ';', low_memory=False)

# sex column
df_client['month'] = df_client.birth_number.astype(str).str[2:4].astype(int)
df_client['sex'] = df_client['month'].apply(lambda x: 'f' if x>=50 else 'm')

# fix month display
df_client['birth_date'] = np.where(df_client['sex'] != 'm', df_client['birth_number']-5000 , df_client['birth_number'])

# date conversion
df_client.birth_date = pd.to_datetime(df_client['birth_date'], format='%y%m%d')

# drop columns
df_client = df_client.drop(["month", "birth_number"], axis=1)

df_client

,client_id,district_id,sex,birth_date
0,1,18,f,1970-12-13
1,2,1,m,2045-02-04
2,3,1,f,2040-10-09
3,4,5,m,2056-12-01
4,5,5,f,2060-07-03
...,...,...,...,...
5364,13955,1,f,2045-10-30
5365,13956,1,m,2043-04-06
5366,13968,61,m,2068-04-13
5367,13971,67,f,2062-10-19


## Disposition

In [5]:
# load data
df_disposition = pd.read_csv(r"Data\csv\disp.csv", sep = ';', low_memory=False)

# translate values
df_disposition.type = df_disposition.type.replace({'DISPONENT': 'user',
                                                   'OWNER': 'owner'})
# rename columns
df_disposition = df_disposition.rename({'disp_id': 'disposition_id',
                                        'type': 'disposition_type'}, axis=1) 

df_disposition

,disposition_id,client_id,account_id,disposition_type
0,1,1,1,owner
1,2,2,2,owner
2,3,3,2,user
3,4,4,3,owner
4,5,5,3,user
...,...,...,...,...
5364,13647,13955,11349,owner
5365,13648,13956,11349,user
5366,13660,13968,11359,owner
5367,13663,13971,11362,owner


## District

In [6]:
# load data
df_district = pd.read_csv(r"Data\csv\district.csv", sep = ';', low_memory=False)

# replace values
df_district.A2 = df_district.A2.replace({'Hl.m. Praha': 'Prague'})

# rename columns
df_district = df_district.rename({'A1': 'district_id',
                                  'A2': 'district_name',
                                  'A3': 'region_name',
                                  'A4': 'number_of_inhabitants',
                                  'A5': 'municipalities_499',
                                  'A6': 'municipalities_1499',
                                  'A7': 'municipalities_9999',
                                  'A8': 'municipalities_10000',
                                  'A9': 'number_of_cities',
                                  'A10': 'pct_urban',
                                  'A11': 'avg_salary',
                                  'A12': 'unemployment_1995',
                                  'A13': 'unemployment_1996',
                                  'A14': 'entrepreneurs_per_1000_inhabitants',
                                  'A15': 'number_of_crimes_1995',
                                  'A16': 'number_of_crimes_1996'}, axis=1)

# Add coordinates
geolocator = Nominatim(user_agent="Your_Name")
city_dict = {"city":[],
             "lat":[],
             "lon":[]}
for cities in df_district.district_name.unique():
    location = geolocator.geocode(cities)
    city_dict["city"].append(cities)
    city_dict["lat"].append(location.latitude)
    city_dict["lon"].append(location.longitude)
city_geo = pd.DataFrame(city_dict)

# merge
df_district = df_district.merge(city_geo, left_on='district_name', right_on='city')
df_district = df_district.drop(["city"], axis=1)

df_district

,district_id,district_name,region_name,number_of_inhabitants,municipalities_499,municipalities_1499,municipalities_9999,municipalities_10000,number_of_cities,pct_urban,avg_salary,unemployment_1995,unemployment_1996,entrepreneurs_per_1000_inhabitants,number_of_crimes_1995,number_of_crimes_1996,lat,lon
0,1,Prague,Prague,1204953,0,0,0,1,1,100.0,12541,0.29,0.43,167,85677,99107,50.087465,14.421254
1,2,Benesov,central Bohemia,88884,80,26,6,2,5,46.7,8507,1.67,1.85,132,2159,2674,49.781892,14.686912
2,3,Beroun,central Bohemia,75232,55,26,4,1,5,41.7,8980,1.95,2.21,111,2824,2813,49.964029,14.073391
3,4,Kladno,central Bohemia,149893,63,29,6,2,6,67.4,9753,4.64,5.05,109,5244,5892,50.147850,14.102538
4,5,Kolin,central Bohemia,95616,65,30,4,1,6,51.4,9307,3.85,4.43,118,2616,3040,50.028889,15.201157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,73,Opava,north Moravia,182027,17,49,12,2,7,56.4,8746,3.33,3.74,90,4355,4433,49.938900,17.902417
73,74,Ostrava - mesto,north Moravia,323870,0,0,0,1,1,100.0,10673,4.75,5.44,100,18782,18347,49.834914,18.282008
74,75,Prerov,north Moravia,138032,67,30,4,2,5,64.6,8819,5.38,5.66,99,4063,4505,49.455377,17.450862
75,76,Sumperk,north Moravia,127369,31,32,13,2,7,51.2,8369,4.73,5.88,107,3736,2807,49.965552,16.970565


## Loan

In [7]:
# load data
df_loan = pd.read_csv(r"Data\csv\loan.csv", sep = ';', low_memory=False)

# date conversion
df_loan.date = pd.to_datetime(df_loan.date, format='%y%m%d')

# replace values
df_loan.status = df_loan.status.replace({'A': 'closed',
                                         'B': 'closed_default',
                                         'C': 'active',
                                         'D': 'active_default',})

# rename columns
df_loan = df_loan.rename({'date': 'loan_date',
                          'duration': 'loan_duration',
                          'status': 'loan_status',
                          'payments': 'loan_payments',
                          'amount': 'loan_amount'}, axis=1)

df_loan

,loan_id,account_id,loan_date,loan_amount,loan_duration,loan_payments,loan_status
0,5314,1787,1993-07-05,96396,12,8033.0,closed_default
1,5316,1801,1993-07-11,165960,36,4610.0,closed
2,6863,9188,1993-07-28,127080,60,2118.0,closed
3,5325,1843,1993-08-03,105804,36,2939.0,closed
4,7240,11013,1993-09-06,274740,60,4579.0,closed
...,...,...,...,...,...,...,...
677,4989,105,1998-12-05,352704,48,7348.0,active
678,5221,1284,1998-12-05,52512,12,4376.0,active
679,6402,6922,1998-12-06,139488,24,5812.0,active
680,5346,1928,1998-12-06,55632,24,2318.0,active


## Order

In [8]:
# load data
df_order = pd.read_csv(r"Data\csv\order.csv", sep = ';', low_memory=False)

# drop column
df_order = df_order.drop(["k_symbol"], axis=1)

# rename columns
df_order = df_order.rename({'account_to': 'recipient_account',
                            'bank_to': 'partner_bank',
                            'amount': 'order_amount'}, axis=1)
df_order

,order_id,account_id,partner_bank,recipient_account,order_amount
0,29401,1,YZ,87144583,2452.0
1,29402,2,ST,89597016,3372.7
2,29403,2,QR,13943797,7266.0
3,29404,3,WX,83084338,1135.0
4,29405,3,CD,24485939,327.0
...,...,...,...,...,...
6466,46334,11362,YZ,70641225,4780.0
6467,46335,11362,MN,78507822,56.0
6468,46336,11362,ST,40799850,330.0
6469,46337,11362,KL,20009470,129.0


## Transactions

In [9]:
# load data
df_transactions = pd.read_csv(r"Data\csv\trans.csv", sep = ';', low_memory=False)

# date conversion
df_transactions.date = pd.to_datetime(df_transactions.date, format='%y%m%d')

# translate values
df_transactions.type = df_transactions.type.replace({'PRIJEM': 'credit',
                                                     'VYDAJ': 'withdrawal',
                                                     'VYBER': 'withdrawal'})

# translate values
df_transactions.operation = df_transactions.operation.replace({'VYBER KARTOU': 'card_withdrawal',
                                                               'VKLAD': 'credit_in_cash',
                                                               'PREVOD Z UCTU': 'collection_from_another_bank',
                                                               'VYBER': 'withdrawal_in_cash',
                                                               'PREVOD NA UCET': 'remittance_to_another_bank'})

# translate values
df_transactions.k_symbol = df_transactions.k_symbol.replace({'POJISTNE': 'insurance_payment',
                                                             'SLUZBY': 'payment_on_statement',
                                                             'UROK': 'interest_credited',
                                                             'SANKC. UROK': 'sanction_interest',
                                                             'SIPO': 'household',
                                                             'DUCHOD': 'old_age_pension',
                                                             'UVER': 'loan_payment',
                                                            ' ':'unknown'})

# rename columns
df_transactions = df_transactions.rename({'trans_id': 'transaction_id',
                                          'account': 'target_account',
                                          'amount': 'transaction_amount',
                                          'balance': 'account_balance',
                                          'operation': 'operation_type',
                                          'date': 'transaction_date',
                                          'type': 'transaction_type',
                                          'bank': 'partner_bank'}, axis=1)

df_transactions

,transaction_id,account_id,transaction_date,transaction_type,operation_type,transaction_amount,account_balance,k_symbol,partner_bank,target_account
0,695247,2378,1993-01-01,credit,credit_in_cash,700.0,700.0,NaN,NaN,NaN
1,171812,576,1993-01-01,credit,credit_in_cash,900.0,900.0,NaN,NaN,NaN
2,207264,704,1993-01-01,credit,credit_in_cash,1000.0,1000.0,NaN,NaN,NaN
3,1117247,3818,1993-01-01,credit,credit_in_cash,600.0,600.0,NaN,NaN,NaN
4,579373,1972,1993-01-02,credit,credit_in_cash,400.0,400.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1056315,3626622,2906,1998-12-31,credit,NaN,62.3,13729.4,interest_credited,NaN,NaN
1056316,3627616,2935,1998-12-31,credit,NaN,81.3,19544.9,interest_credited,NaN,NaN
1056317,3625403,2869,1998-12-31,credit,NaN,60.2,14638.2,interest_credited,NaN,NaN
1056318,3626683,2907,1998-12-31,credit,NaN,107.5,23453.0,interest_credited,NaN,NaN


In [10]:
# Connect to database
conn=sqlite3.connect(r'Data\database.db')

In [11]:
df_account.to_sql("accounts", conn, if_exists='replace', index=False)
df_card.to_sql("cards", conn, if_exists='replace', index=False)
df_client.to_sql("clients", conn, if_exists='replace', index=False)
df_disposition.to_sql("dispositions", conn, if_exists='replace', index=False)
df_district.to_sql("districts", conn, if_exists='replace', index=False)
df_loan.to_sql("loans", conn, if_exists='replace', index=False)
df_order.to_sql("orders", conn, if_exists='replace', index=False)
df_transactions.to_sql("transactions", conn, if_exists='replace', index=False)